# RAG Post Retrieval Optimization - Reranking featuring Bedrock (Amazon Nova Pro) and llamaindex

Reranking is a crucial post-retrieval process that refines the initial set of retrieved documents to improve the relevance and quality of information used for generating responses. After the initial retrieval step, which typically uses methods like vector similarity search, reranking takes the retrieved documents and re-scores their relevance to the query using more sophisticated techniques. These may include cross-encoders, BERT-based models, or other advanced algorithms that can better capture semantic nuances and context. Based on these scores, the documents are reordered, pushing the most relevant and informative ones to the top, thereby enhancing the accuracy and contextual appropriateness of the final response.

In this lab, we will highlight the usage of an existing LLM (for example Bedrock Amazon Nova Pro) as a reranker model to improve the relevance of the documents retrieved from Amazon SEC filings. We also utilized the Bedrock Amazon Nova Pro model and the Bedrock Titan Text Embedding v2.0 model for natural language processing and document embeddings to build this RAG system. The key steps were: ingesting PDF documents into a vector database index, retrieving top relevant nodes based on a query using semantic search, reranking the retrieved nodes using Bedrock Nova Pro LLM and synthesizing a final response consolidating the reranked information. See below image for details.

- Vector Database (Faiss / local on the notebook for this demo)
- LLM (Amazon Bedrock - Amazon Nova Pro)
- Embeddings Model (Bedrock Titan Text Embedding v2.0)
- ReRanking Model (Amazon Nova Pro) running on Bedrock
- Llamaindex for orchestration (ingestion, reranking, retrieval and final response synthesis)
- Datasets (Amazon SEC 10-k statements for year 2022 and 2023 )

<img src="reranking-bedrock.png" width="800" height="400">

## Pre-req
You must run the `[workshop_setup.ipynb]`(../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [1]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook") 

/tmp/ipykernel_3201/952814476.py:2: UserWarning: Warning: if you did not run lab00-setup, please go back and run the lab00 notebook
  warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook")



### > Setup
We start by importing necessary llamaindex libraries

In [2]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import Settings

We select Anthropic Claude3 Sonnet as our LLM. For embedding model, we are selecting Amazon Titan Text Embed v2.0. Chunk size is set at 512 for this example.

In [6]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding, Models
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
#from llama_index.core.postprocessor.bedrock_rerank import AWSBedrockRerank

profile_name = "default"

# define the LLM
llm = BedrockConverse(
    model="us.amazon.nova-pro-v1:0",
    profile_name=profile_name,
)

# define the embedding model
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

### > Document Ingestion
We ingest and index the data stored in data directory. The amazon folder has SEC-10k files from 2022 and 2023.

In [9]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="../data/lab03/amazon/").load_data()

# build index
amazon_index = VectorStoreIndex.from_documents(
    amazon_secfiles,
    use_async=True,
)

### > Helper functions
Helper functions to help retrieve and visualize relevant nodes from vector database based on a given query. It allows for optional reranking of results using a language model (in this case, Amazon Nova Pro). The retrieved nodes are then displayed in a formatted HTML table.

In [10]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML

def get_retrieved_nodes(
    query_str, vector_top_k=5, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=amazon_index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = RankGPTRerank(
            llm=BedrockConverse(model = "us.amazon.nova-pro-v1:0", profile_name="default"),
            top_n=reranker_top_n,
            verbose=True,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

### > Test retrieval without reranking

In [11]:
new_nodes = get_retrieved_nodes(
    "Describe key business risks for Amazon during covid",
    vector_top_k=10,
    with_reranker=False,
)

In [12]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.728445,"These risk factors do not identify all risks that we face; our operations could also be affected by factors, events, oruncertainties that are not presently known to us or that we currently do not consider to present significant risks to our operations. In addition to the factorsdiscussed in Item 7 of Part II, “Management’s Discussion and Analysis of Financial Condition and Results of Operations,” and in the risk factors below, globaleconomic and geopolitical conditions and additional or unforeseen circumstances, developments, or events may give rise to or amplify many of the risksdiscussed below. Many of the risks discussed below also impact our customers, including third-party sellers, which could indirectly have a material adverseeffect on us.Business and Industry RisksWe Face Intense CompetitionOur businesses are rapidly evolving and intensely competitive, and we have many competitors across geographies, including cross-border competition,and in different industries, including physical, e-commerce, and omnichannel retail, e-commerce services, web and infrastructure computing services,electronic devices, digital content, advertising, grocery, and transportation and logistics services. Some of our current and potential competitors have greaterresources, longer histories, more customers, and/or greater brand recognition, particularly with our newly-launched products and services and in our newergeographic regions. They may secure better terms from vendors, adopt more aggressive pricing, and devote more resources to technology, infrastructure,fulfillment, and marketing.Competition continues to intensify, including with the development of new business models and the entry of new and well-funded competitors, and asour competitors enter into business combinations or alliances and established companies in other market segments expand to become competitive with ourbusiness. In addition, new and enhanced technologies, including search, web and infrastructure computing services, practical applications of artificialintelligence and machine learning, digital content, and electronic devices continue to increase our competition. The internet facilitates competitive entry andcomparison shopping, which enhances the ability of new, smaller, or lesser known businesses to compete against us."
1,0.728156,"These risk factors do not identify all risks that we face; our operations could also be affected by factors, events, oruncertainties that are not presently known to us or that we currently do not consider to present significant risks to our operations. In addition to the factorsdiscussed in Item 7 of Part II, “Management’s Discussion and Analysis of Financial Condition and Results of Operations—Overview,” and in the risk factorsbelow, global economic and geopolitical conditions and additional or unforeseen circumstances, developments, or events may give rise to or amplify many ofthe risks discussed below. Many of the risks discussed below also impact our customers, including third-party sellers, which could indirectly have a materialadverse effect on us.Business and Industry RisksWe Face Intense CompetitionOur businesses are rapidly evolving and intensely competitive, and we have many competitors across geographies, including cross-border competition,and in different industries, including physical, e-commerce, and omnichannel retail, e-commerce services, web and infrastructure computing services,electronic devices, digital content, advertising, grocery, and transportation and logistics services. Some of our current and potential competitors have greaterresources, longer histories, more customers, and/or greater brand recognition, particularly with our newly-launched products and services and in our newergeographic regions. They may secure better terms from vendors, adopt more aggressive pricing, and devote more resources to technology, infrastructure,fulfillment, and marketing.Competition continues to intensify, includi

### > Add reranking
We will use a llamaIndex module called `RankGPTRerank`. This is a zero-shot listwise passage reranking method that uses large language models (Amazon Nova Pro) to efficiently reorder retrieved passages based on their relevance to a given query.

In [15]:
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

new_nodes = get_retrieved_nodes(
    "Describe key business risks for Amazon during covid",
    vector_top_k=10,
    reranker_top_n=5,
    with_reranker=True,
)

After Reranking, new rank list for nodes: [4, 2, 9, 6, 8, 3, 0, 1, 5, 7]

In [16]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.662429,"fulfillment, and other expense categories;• availability of and increases in the prices of transportation (including fuel), resources such as land, water, and energy, commodities like paper andpacking supplies and hardware products, and technology infrastructure products, including as a result of inflationary pressures;• constrained labor markets, which increase our payroll costs;• the extent to which operators of the networks between our customers and our stores successfully charge fees to grant our customers unimpaired andunconstrained access to our online services;• our ability to collect amounts owed to us when they become due;• the extent to which new and existing technologies, or industry trends, restrict online advertising or affect our ability to customize advertising orotherwise tailor our product and service offerings;• the extent to which use of our services is affected by spyware, viruses, phishing and other spam emails, denial of service attacks, data theft, computerintrusions, outages, and similar events;• the extent to which we fail to maintain our unique culture of innovation, customer obsession, and long-term thinking, which has been critical to ourgrowth and success;• disruptions from natural or human-caused disasters (including public health crises) or extreme weather (including as a result of climate change),geopolitical events and security issues (including terrorist attacks, armed hostilities, and political conflicts, including those involving China), labor ortrade disputes (including restrictive governmental actions impacting us, our customers, and our third-party sellers and suppliers in China or otherforeign countries), and similar events; and• potential negative impacts of climate change, including: increased operating costs due to more frequent extreme weather events or climate-relatedchanges, such as rising temperatures and water scarcity; increased investment requirements associated with the transition to a low-carbon economy;decreased demand for our products and services as a result of changes in customer behavior; increased compliance costs due to more extensive andglobal regulations and third-party requirements; and reputational damage resulting from perceptions of our environmental impact."
1,0.712217,"which has been critical to ourgrowth and success;• disruptions from natural or human-caused disasters (including public health crises) or extreme weather (including as a result of climate change),geopolitical events and security issues (including terrorist attacks, armed hostilities, and political conflicts, including those involving China), labor ortrade disputes (including restrictive governmental actions impacting us, our customers, and our third-party sellers and suppliers in China or otherforeign countries), and similar events; and• potential negative impacts of climate change, including: increased operating costs due to more frequent extreme weather events or climate-relatedchanges, such as rising temperatures and water scarcity; increased investment requirements associated with the transition to a low-carbon economy;decreased demand for our products and services as a result of changes in customer behavior; increased compliance costs due to more extensive andglobal regulations and third-party requirements; and reputational damage resulting from perceptions of our environmental impact.We Face Risks Related to Successfully Optimizing and Operating Our Fulfillment Network and Data CentersFailures to adequately predict customer demand and consumer spending patterns or otherwise optimize and operate our fulfillment network and datacenters successfully from time to time result in excess or insufficient fulfillment or data center capacity, service interruptions, increased costs, and impairmentcharges, any of which could materially harm our business. As we continue to add fulfillment and data center capability or add new businesses with differentrequirements, our fulfillment

### > Apply reranking to final response generation

In [17]:
query_engine_naive = amazon_index.as_query_engine(similarity_top_k=10)

In [18]:
response = query_engine_naive.query(
    "Describe key business risks for Amazon during covid"
)

In [19]:
reranker = RankGPTRerank(
            llm=BedrockConverse(model = "us.amazon.nova-pro-v1:0", profile_name="default"),
            top_n=5,
            verbose=True,
        )

In [20]:
query_engine_rerank = amazon_index.as_query_engine(similarity_top_k=10, node_postprocessor =[reranker])

In [21]:
response = query_engine_rerank.query(
    "Describe key business risks for Amazon during covid"
)

### > Final reponse

In [22]:
print(response)

Key business risks for Amazon during the COVID-19 pandemic included disruptions from natural or human-caused disasters, extreme weather, geopolitical events, security issues, labor or trade disputes, and potential negative impacts of climate change. Additionally, the company faced risks related to successfully optimizing and operating its fulfillment network and data centers, including failures to predict customer demand, service interruptions, increased costs, and impairment charges. The loss of key senior management personnel or the failure to hire and retain highly skilled and other personnel also posed significant risks. Furthermore, Amazon's supplier relationships and commercial agreements exposed it to additional risks, such as limited or single-sources of supply, and the inability to procure alternatives in a timely and efficient manner.
